In [125]:
import numpy as np
import torch
a = [[[1.,2],[3.,4],[5.,6]],[[6.,7],[8.,9],[10.,11]]]
b = [[[9.,9.],[9.,9.],[9.,9.]],[[8.,8.],[8.,8.],[8.,8.]]]
a = torch.Tensor(a)
b = torch.Tensor(b)
#print(np.concatenate((a, b), axis=1))
a = torch.FloatTensor(a)
b = torch.FloatTensor(b)
print(a.size())
print(b.size())
print(torch.cat([a,b], dim=0))
print(torch.cat([a,b], dim=-1).size())
c = torch.randn(4, 4)
print(c)
c.view([2,-1,4])
a = torch.Tensor([[1,2,3,4],[5,6,7,8]])
b = a.clone()
for i, row in enumerate(b):
    b[i][0] = 1
print(a)
print(b)

torch.Size([2, 3, 2])
torch.Size([2, 3, 2])
tensor([[[ 1.,  2.],
         [ 3.,  4.],
         [ 5.,  6.]],

        [[ 6.,  7.],
         [ 8.,  9.],
         [10., 11.]],

        [[ 9.,  9.],
         [ 9.,  9.],
         [ 9.,  9.]],

        [[ 8.,  8.],
         [ 8.,  8.],
         [ 8.,  8.]]])
torch.Size([2, 3, 4])
tensor([[-1.7508, -0.4469, -0.5798, -0.1931],
        [-0.7578, -1.5895, -0.6888,  0.1251],
        [ 0.1907,  1.3085, -0.9141, -0.6718],
        [-0.3653, -1.0359, -0.5003,  1.0438]])
tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.]])
tensor([[1., 2., 3., 4.],
        [1., 6., 7., 8.]])


In [123]:
a = torch.Tensor([torch.tensor([3,4]), torch.tensor([1,2,3,4]), torch.tensor([1])])
a1 = torch.Tensor([torch.tensor([3,4]), torch.tensor([1,2,3]), torch.tensor([1])])
b = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)
b1 = torch.nn.utils.rnn.pad_sequence(a1, batch_first=True)
x = torch.nn.utils.rnn.pad_sequence(torch.cat([a.data.tolist(),a1.data.tolist(),dim=0]), batch_first=True)
print(a)
print(a1)
print(b)
print(b1)
perm_idx = torch.LongTensor([1,0,2])
_, desort = torch.sort(perm_idx, descending=False)
desort = desort.type(torch.long)
print(perm_idx)
print(desort)

ValueError: only one element tensors can be converted to Python scalars

In [156]:
sorted_b = torch.LongTensor([[1,2,3,4], [3,4,0,0],[1,0,0,0]])
sorted_b1 = torch.LongTensor([[1,2,3,0], [3,4,0,0],[1,0,0,0]])
c = torch.nn.utils.rnn.pack_padded_sequence(sorted_b, batch_first=True, lengths=[4,4,1])
print("c!",c.data[c.data.nonzero()].view([1,-1])[0])
print("c",c)
#c.data = c.data[c.data.nonzero()].view([1,-1])[0].clone()
#c1 = torch.nn.utils.rnn.pack_padded_sequence(sorted_b1, batch_first=True, lengths=[3,2,1])
#print(c1)
#lstm = torch.nn.LSTM(char_lstm_input_dim, self.config.char_lstm_output_dim, 1, bidirectional=True)
d, _ = torch.nn.utils.rnn.pad_packed_sequence(c, batch_first=True)
d1, _ = torch.nn.utils.rnn.pad_packed_sequence(c1, batch_first=True)
print(d)
e = d[desort]
e1 = d1[desort]
print(e)
print(e1)

c! tensor([1, 3, 1, 2, 4, 3, 4])
c PackedSequence(data=tensor([1, 3, 1, 2, 4, 3, 0, 4, 0]), batch_sizes=tensor([3, 2, 2, 2]))
tensor([[1, 2, 3, 4],
        [3, 4, 0, 0],
        [1, 0, 0, 0]])
tensor([[3, 4, 0, 0],
        [1, 2, 3, 4],
        [1, 0, 0, 0]])
tensor([[3, 4, 0],
        [1, 2, 3],
        [1, 0, 0]])


In [92]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import itertools
import string

"""
Since character embeddings are a bit weak in pytorch 3, this will hopefully help out
I think these should be trainable and also, invertable!
So you can actually recover output from the embeddings using Cos Similarity
"""

class CharacterEmbedding:
    def __init__(self, embedding_size):

        self.vocab = ['<pad>'] + list(string.printable) + ['<SOS>', '<EOS>']
        self.embed = nn.Embedding(len(self.vocab), embedding_size)
        self.is_cuda = False
        self.cos = nn.CosineSimilarity(dim=2)

    def flatten(self, l):
        return list(itertools.chain.from_iterable(l))

    def embedAndPack(self, seqs, batch_first=False):

        vectorized_seqs = [[self.vocab.index(tok) for tok in seq]for seq in seqs]

        # get the length of each seq in your batch
        seq_lengths = torch.LongTensor(list(map(len, vectorized_seqs)))
        seq_lengths = seq_lengths.cuda() if self.is_cuda else seq_lengths

        # dump padding everywhere, and place seqs on the left.
        # NOTE: you only need a tensor as big as your longest sequence
        seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long()
        seq_tensor = seq_tensor.cuda() if self.is_cuda else seq_tensor

        for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
            seq_tensor[idx, :seqlen] = torch.LongTensor(seq).cuda() if self.is_cuda else torch.LongTensor(seq)

        # SORT YOUR TENSORS BY LENGTH!
        seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
        seq_tensor = seq_tensor[perm_idx]

        # utils.rnn lets you give (B,L,D) tensors where B is the batch size, L is the maxlength, if you use batch_first=True
        # Otherwise, give (L,B,D) tensors
        if not batch_first:
            seq_tensor = seq_tensor.transpose(0,1) # (B,L,D) -> (L,B,D)

        # embed your sequences
        seq_tensor = self.embed(seq_tensor)

        # pack them up nicely
        return pack_padded_sequence(seq_tensor, seq_lengths.cpu().numpy(), batch_first=True)

    def cuda(self):
        self.is_cuda = True
        self.embed = self.embed.cuda()
        return self

    def unpackToSequence(self, packed_output):
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        words = self.unembed(output)
        return words

    def unembed(self, embedded_sequence):
        weights = self.embed.state_dict()['weight']
        weights = weights.transpose(0,1).unsqueeze(0).unsqueeze(0)
        e_sequence = embedded_sequence.unsqueeze(3).data
        cosines = self.cos(e_sequence, weights)
        _, indexes = torch.topk(cosines, 1, dim=2)

        words = []
        for word in indexes:
            word_l = ''
            for char_index in word:
                word_l += self.vocab[char_index[0]]
            words.append(word_l)
        return words









if __name__ == '__main__':

    seqs = ['ghatmasala','nicela','c-pakodas']

    # make model

    embedding = CharacterEmbedding(embedding_size=5)

    lstm = nn.LSTM(5, 5, batch_first=True)

    packed_input = embedding.embedAndPack(seqs, batch_first=True)
    words = embedding.unpackToSequence(packed_input)

    print(words)

    # throw them through your LSTM (remember to give batch_first=True here if you packed with it)
    packed_output, (ht, ct) = lstm(packed_input)

    words = embedding.unpackToSequence(packed_output)


    print(words)

['ghatmasala', 'c-pakodas<EOS>', 'nicela<EOS><EOS><EOS><EOS>']
['<EOS>v|<pad>r|M|z,', 'z*||^^J6,<EOS>', '\rDcP\tS<EOS><EOS><EOS><EOS>']


In [ ]:
def get_char_sequence(model, batch_char_index_matrices, batch_word_len_lists):
    
    # Given an input of the size [2,7,14], we will convert it a minibatch of the shape [14,14] to 
    # represent 14 words(7 in each sentence), and 14 characters in each word.
    ## NOTE: Please DO NOT USE for Loops to iterate over the mini-batch.

    batch_size=len(batch_word_len_lists)
    max_word_len=max([max(i) for i in batch_word_len_lists])
    max_sent_len=max([len(i) for i in batch_word_len_lists])
    print(max_word_len)
    
    
    
    #1. Reshape
    _14_14_resize= batch_char_index_ex_matrices.resize(bat(batch_size*max_sent_len, max_word_len)
    _14_resize= batch_word_len_en_lists.resize(bat(batch_size*max_sent_len)
    print(_14_14_resize.shape)

 )

    
    # 2. Get corresponding char_Embeddings, we will have a Final Tensor of the shape [14, 14, 50]
    input_char_embeds =  = model.char_emb_embeds(_14_14_resize)
    input_embeds=input_char_embeds
    print(input_ut_embeds.shape)
  )
    
    
    #  # 3.Sort the the mini-batch wrt word-lengths, to form a pack_padded sequence.
    perm_idx, sorted_batch_word_len_list =  = model.sort_inp_input(_14_resize)
    sorted_input_embeds = input_embeds[perm_idx]
    _, desorted_indices =  = torch.sort(per(perm_idx, descending=False)
    
    output_sequence = pack_padded_sequence(sorted_input_embeds, lengths=sorted_batch_word_len_en_list.data.tolist(), (), batch_first=True)

    
    #  # 4.Feed the the pack_padded sequence to the char_LSTM layer.
    output_sequence, state =  = model.char_lst_lstm(output_sequence)
    
    
    # 5. Get hidden state of the shape [2,14,50].
    output_sequence= pad_packed_sequence(output_sequence, batch_first=True)
    hidden_state=state[0]   #hidden, cell= state
    
    # 6. Recover the hidden_states corresponding to the sorted index.
    result= hidden_state[:,desorted_indices]


    # 7. Re-shape it to get a Tensor the shape [2,7,100].
    allcat=[]
    
    for batch in range(0, batch_size):
        batch_concat=[]
        begin=batch*max_sent_len
        end= max_sent_len*(batch+1)
        print(begin, end)
        for word_len in range(begin, end):
            direction_cat= t= torch.cat((re((result[0][word_len], result[1][word_len]),0)
            batch_ch_concat.append(dir(direction_on_cat.detach().n().numpy())
           allcat.append(bat(batch_concat)

    allcat =  = torch.tensor(all(allcat)
        
    return allcat